In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Module 3.1- Run LandTrendr over Composites

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/2-LandTrendr.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/3-LandTrendr.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/3-LandTrendr.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview

LandTrendr, short for Landsat-based detection of Trends in Disturbance and Recovery, is a set of temporal segmentation algorithms that that are used in change detection. LandTrendr also removes noise from interannual spectral data. You can use the change detection outputs and the smoothed spectral data that LandTrendr produces as inputs to modeling software, like LCMS. 

**This notebook uses Landsat and Sentinel-2 composites, like the ones you created in the previous notebook, as inputs to the LandTrendr temporal segmentation and spectral smoothing operations.**

##### To learn more about LandTrendr:
- [LandTrendr Guide](https://emapr.github.io/LT-GEE/)`
- [LandTrendr GEE Publication](https://www.mdpi.com/2072-4292/10/5/691)

##### To learn more about the LandTrendr implementation in geeViz:
- [LandTrendr code in geeViz](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/changeDetectionLib.py#L183)
- Get used to referring back to this documentation throughout this module when you want or need clarification on how a function works, or what it's doing!

### Objective

In this tutorial, you learn how to create and manipulate LandTrendr outputs.

**This tutorial uses the following Google Cloud services:**

- Google Earth Engine
- Google Cloud Platform

**The steps performed include:**

- Create LandTrendr outputs
- Manipulate EE array image objects to get meaningful data out of LandTrendr outputs

**Learning objectives include:**
- Users will understand the purpose of temporal segmentation as implemented in LandTrendr.
- Users will understand key parameters in LandTrendr algorithm. 
- Users will be able to generate and manipulate array outputs from LandTrendr. 


## Before you begin

### Set your current URL under `workbench_url`
This gives the Map Viewer a url in which to host the viewer we will be generating. 
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/` (See the image below)

![workspace url](img/workspace-url.png)

### Set a folder to use for all exports under `export_path_root` 
This folder should be an assets folder in an existing GEE project. 
* It will be something like `projects/projectID/assets/someFolder`
* This folder does not have to already exist. If it does not exist, it will be created

In [1]:
workbench_url = 'https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/'
export_path_root  = 'projects/rcr-gee/assets/lcms-training'

# set specific import path  for assets that will need to be loaded / prebaked for each module when this lab is done in Qwiklabs
import_path_prebaked = 'projects/rcr-gee/assets/lcms-training'
print('Done')

Done


### Installation
First, install necessary Python packages. Uncomment the first line to upgrade geeViz if necessary.

In [2]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml

import inspect

ee = getImagesLib.ee
Map = getImagesLib.Map

print('Done')

Initializing GEE
Successfully initialized
geeViz package folder: /opt/conda/lib/python3.10/site-packages/geeViz
Done


### Set up your work environment

Create a folder in your export path where you will export the composites. In addition, create a blank image collection where your composites will live.

Currently, when running within Colab, geeView uses a different project to authenticate through, so you may need to make your asset public to view from within Colab.

In [3]:
# Create folder and a collection and make public

export_composite_collection = f'{export_path_root}/lcms-training_module-2_composites'
export_landTrendr_collection = f'{export_path_root}/lcms-training_module-3_landTrendr'

aml.create_asset(export_landTrendr_collection,asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(export_landTrendr_collection,writers = [],all_users_can_read = True,readers = [])

# aml.batchCopy('projects/rcr-gee/assets/landTrendr-lcms-training-module-2',export_landTrendr_collection,outType = 'imageCollection')
# aml.batchDelete(export_landTrendr_collection)
print('Done')

Found the following sub directories:  ['lcms-training', 'lcms-training_module-3_landTrendr']
Will attempt to create them if they do not exist
Asset projects/rcr-gee/assets/lcms-training already exists
Asset projects/rcr-gee/assets/lcms-training/lcms-training_module-3_landTrendr already exists
Updating permissions for:  projects/rcr-gee/assets/lcms-training/lcms-training_module-3_landTrendr
Done


## Change detection with LandTrendr - intro

## Introduction to LandTrendr

LandTrendr is a suite of temporal segmentation algorithm developed by the eMapr Lab at Oregon State University. 

LandTrendr works by taking a time series of annual composite images and fitting multiple linear segments that describe the spectral trajectory. The resulting segments can be used to describe change processes on the landscape. 

![landtrendr_pixel](img/module-3-1/LandTrendr_pixel_story)*Every pixel tells a story. Landsat provides a historical record of the character of landscapes. By extracting a single pixel from a time series of Landsat imagery, it is possible to recount the state and change of the features composing the 1-hectare area of a pixel through time. In this example, we analyze the history of a conifer forest pixel from an industrial forest region of the Pacific Northwest (USA) that experiences a period of relative stability, a dramatic, rapid loss of vegetation, and subsequent regeneration. Figure and caption from the [LT-GEE Guide](https://emapr.github.io/LT-GEE/landtrendr.html).*

For example, a short, steep decline is likely to be a fast change, like harvest or fire. A long, shallow decline is likely to be a slower change, due to insects, disease, or drought. LandTrendr can also track periods of recovery, like a long shallow incline following a harvest or fire event. 

![landtrendr_segmentation](img/module-3-1/LandTrendr-segmentation.png)*LandTrendr pixel time series segmentation. Image data is reduced to a single band or spectral index and then divided into a series of straight line segments by breakpoint (vertex) identification.Figure and caption from the [LT-GEE Guide](https://emapr.github.io/LT-GEE/landtrendr.html).*

### Bands used in LandTrendr

LandTrendr relies on a time series of annual composites, one image per year. You prepared this series of annual composites in the previous lab! LandTrendr can take a variety of spectral bands and indices as input. 

NBR, NDMI, and NDVI tend to work best. Other good options are Tasseled-Cap wetness and tcAngleBG. But, you must include 'red','nir','swir1' in order to visualize as false color composites, like we'll do later on in the lab. 

Run the code below to view an example of composite data input to LandTrendr data. Look at the bands included in the **actual** collection that LCMS uses for LandTrendr in PuertoRico.



### Example 
View an example of data input to LandTrendr data -- don't worry about the params too much yet. 

In [4]:
# set up map
Map.clearMap()
Map.proxy_url = workbench_url

#### LandTrendr bands and parameters

Lets look at the bands included in the **actual** collection that LCMS uses for LandTrendr in PuertoRico

In [5]:
# Load LCMS data used for Puerto Rico Mapping
lcms_actual_lt_collection = ee.ImageCollection('projects/lcms-292214/assets/R8/PR_USVI/Base-Learners/LandTrendr-Collection-1984-2020')
print('All bands LCMS uses for LandTrendr:',lcms_actual_lt_collection.aggregate_histogram('band').keys().getInfo())

All bands LCMS uses for LandTrendr: ['NBR', 'NDMI', 'NDSI', 'NDVI', 'blue', 'brightness', 'green', 'greenness', 'nir', 'red', 'swir1', 'swir2', 'tcAngleBG', 'wetness']


### Inspect and prepare composites used in LandTrendr

Next, load the composites that you prepared in the previous module and inspect which bands are present in the composites. 

In [6]:
# Load composites
composites = ee.ImageCollection(export_composite_collection)

#Inspect what bands are available by default in composite
print(composites.first().bandNames().getInfo())

['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'compositeObsCount', 'sensor', 'year', 'julianDay']


You'll notice that the composite is missing some of the spectral indices that are present in the composite that LCMS uses. 

In the code block below, you'll perform some image processing on the composites, and you'll also add some additional spectral indices to the composites so that you can use them in the LandTrendr algorithm.

The functions you'll use to add composites are: 
- [`getImagesLib.simpleAddIndices`](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/getImagesLib.py#L1300)
- [`getImagesLib.getTasseledCap`](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/getImagesLib.py#L1340)
[`getImagesLib.simpleAddTCAngles`](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/getImagesLib.py#L1429)

You can click on the name of the function above to view the source code in the geeViz [getImagesLib](https://github.com/gee-community/geeViz/blob/master/getImagesLib.py). 

In [7]:
# Decompress composites by dividing by 10000 for optical bands and add indices
composites = composites.select(['blue','green','red','nir','swir1','swir2']).map(lambda img: img.divide(10000).float().copyProperties(img,['system:time_start']))
# add indices
composites = composites.map(getImagesLib.simpleAddIndices)\
                      .map(getImagesLib.getTasseledCap)\
                      .map(getImagesLib.simpleAddTCAngles)

#inspect
print(composites.first().getInfo())

{'type': 'Image', 'bands': [{'id': 'blue', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [12391, 4871], 'crs': 'PROJCS["Albers_Conical_Equal_Area", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      TOWGS84[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH], \n    AUTHORITY["EPSG","4326"]], \n  PROJECTION["Albers_Conic_Equal_Area"], \n  PARAMETER["central_meridian", -96.0], \n  PARAMETER["latitude_of_origin", 23.0], \n  PARAMETER["standard_parallel_1", 29.5], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["standard_parallel_2", 45.5], \n  UNIT["m", 1.0], \n  AXIS["Easting", EAST], \n  AXIS["Northing", NORTH]]', 'crs_transform': [30, 0, 3039345, 0, -30, 64635]

### Extract parameters from composites

We'll use the parameters from the composites later on in the script. 

In [9]:
#### Load information from composites - this will be used as parameters for LandTrendr and export

# get study area from composite
studyArea = composites.first().geometry()

# get properties of composite images
props = composites.first().toDictionary().getInfo()

startYear = props['startYear']
endYear = props['endYear']

startJulian = props['startJulian']
endJulian = props['endJulian']

proj = composites.first().projection().getInfo()

# Pull out the crs
# Depending on if a wkt or epsg format is used, it will be stored under a different key
if 'crs' not in proj.keys():
    crs = proj['wkt']
else:
    crs = proj['crs']
    
transform = proj['transform']
scale = None



print('Done')

KeyError: 'startYear'

### Inspect

Run the codeblock below to set up the map and add the composites to the map.

Click on the layers on the map to explore the values of the indices that were created. 

In [10]:
# set up map
Map.clearMap()
# reset port if necessary
Map.port = 1236
Map.proxy_url = workbench_url

# add composites
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse,'Composites')

#center map
Map.centerObject(studyArea )

Map.turnOnInspector()
Map.view()

Adding layer: Composites
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Starting local web server at: http://localhost:1236/geeView/
HTTP server command: "/opt/conda/bin/python" -m http.server  1236
Done
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/1236/geeView/?accessToken=None


127.0.0.1 - - [21/Aug/2023 22:01:45] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


### Set parameters for LandTrendr

LandTrendr uses the below parameters. In LCMS, we use the values below and don't adjust the parameters between runs. 


 - `timeSeries`: (ImageCollection) Yearly time-series from which to extract breakpoints. The first band is used to find breakpoints, and all subsequent bands are fitted using those breakpoints.
 - `maxSegments`: (Integer) Maximum number of segments to be fitted on the time series.
 - `spikeThreshold`:  (Float, default: 0.9) Threshold for damping the spikes (1.0 means no damping).
 - `vertexCountOvershoot`: (Integer, default: 3) The initial model can overshoot the maxSegments + 1 vertices by this amount. Later, it will be pruned down to maxSegments + 1.
 - `preventOneYearRecovery`: (Boolean, default: False): Prevent segments that represent one year recoveries.
 - `recoveryThreshold`: (Float, default: 0.25) If a segment has a recovery rate faster than 1/recoveryThreshold (in years), then the segment is disallowed.
 - `pvalThreshold`: (Float, default: 0.1) If the p-value of the fitted model exceeds this threshold, then the current model is discarded and another one is fitted using the Levenberg-Marquardt optimizer.
 - `bestModelProportion`: (Float, default: 0.75) Allows models with more vertices to be chosen if their p-value is no more than (2 - bestModelProportion) times the p-value of the best model.
 - `minObservationsNeeded`: (Integer, default: 6) Min observations needed to perform output fitting.


You can experiment with using different parameters to see how the output differs. 

The [LandTrendr Pixel Time Series Plotter UI](https://emapr.github.io/LT-GEE/ui-applications.html#ui-landtrendr-pixel-time-series-plotter) can also be helpful to see how varying these parameters changes the fitted segments in the output. I recommend you test this in an area / on a pixel where you understand the change history, so that you can better evaluate the line segments fit to the spectral trajectory.

Run the code block below to set the parameters that the LandTrendr algorithm will use. 

In [ ]:
# We can use any/all of the bands, but generally bands that use nir and swir are most useful
bandNames = ['red','nir','swir1','swir2','NBR','NDVI','brightness','greenness','wetness']

#Define landtrendr params
run_params = { \
  'maxSegments':            6,\
  'spikeThreshold':         0.9,\
  'vertexCountOvershoot':   3,\
  'preventOneYearRecovery': False,\
  'recoveryThreshold':      0.25,\
  'pvalThreshold':          0.05,\
  'bestModelProportion':    0.75,\
  'minObservationsNeeded':  6\
}

## Run LandTrendr - on NBR only 

Now let's run LandTrendr over NBR only, as a test example that will allow us to look at the outputs
NBR is the Normalized Burn Ratio
It is sensitive to the presence/absence of moisture


In [ ]:
# run LandTrendr over NBR
test_band = 'NBR'
run_params['timeSeries'] = composites.select([test_band])
raw_LT = ee.Algorithms.TemporalSegmentation.LandTrendr(**run_params)

In [ ]:
### Inspect 

Run the code block below to add the output to the map. Double click on the output on the map to query it. 

In [ ]:
Map.clearMap()
Map.addLayer(raw_LT,{},'LT Raw {}'.format(test_band),True)

Map.turnOnInspector()
Map.view()

Double click on the output on the map to query it. 

Notice this output is not immediately useful for change detection or smoothing out a time series of composites


## Using LandTrendr for Change Detection

Break up workflow below to show intermediate outputs a little better


### Process LandTrendr array outputs

The immediate output of LandTrendr is an array. You have to manipulate these arrays in order to use the data for change detection. 

But, doing this manually will help you understand the way the array outputs are constructed, and will help you gain practice in array manipulation.

Make this so you see what processing is being done for a single pixel, at each step 

You'll be using the functions [`array.Slice`](https://developers.google.com/earth-engine/apidocs/ee-array-slice) and [`array.Mask`](https://developers.google.com/earth-engine/apidocs/ee-array-mask) to manipulate arrays

In [ ]:
# We will now see how LandTrendr can be used on its own for change detection

#### Process LandTrendr array outputs

Make this so you see what processing is being done for a single pixel, at each step 

In [ ]:
# create object representing pixel to query - any px within study area
queryPoint = ee.Geometry.Point([-66.3, 18.42])
geometryBuffer = queryPoint.buffer(1000)

#print(queryPoint.getInfo())
#print(geometryBuffer)

#inspect query point on Map
Map.clearMap()
Map.addLayer(queryPoint,{},'queryPoint',True)
Map.addLayer(geometryBuffer,{},'queryPointBuffer',True)
Map.centerObject(queryPoint)
Map.mapCommandList.append('map.setZoom(12)')
Map.turnOnInspector()
Map.view()

# extract values to px for products 



# plot array outputs to visualize 

In [ ]:
# First, we will select the LandTrendr image array output band
lt_array = raw_LT.select(['LandTrendr'])

# This output has 2 dimensions per pixel
# The rows correspond to 0-years, 1-raw input values, 2-LandTrendr fitted output values, and 3-vertex/non vertex

# First, slice the vertices  and use them as a mask to mask out non vertex values
vertices = lt_array.arraySlice(0,3,4)
lt_array = lt_array.arrayMask(vertices)

In [ ]:
# Look at the outputs
#print(vertices.reduceRegions('mean', queryPoint))
test_array = ee.Array([[1,1,1]],[[2,2,2]])
print(test_array.getInfo())

print(ee.Array([3, 4, 5]).first(ee.Array([0, 1, 2])).getInfo())

#print(lt_array.first(ee.Array([0, 1, 2])).getInfo())

In [ ]:
# In order to perform change detection, we'll need to get the difference between fitted vertex values
# We do this by slicing the first to the next to the last and then the second to the last and subtracting them
left = lt_array.arraySlice(1,0,-1)
right = lt_array.arraySlice(1,1,None)
diff  = left.subtract(right)

# We then slice the right-hand years and the fitted vertex values difference and combine them
years = right.arraySlice(0,0,1)
mag = diff.arraySlice(0,2,3)
forSorting = years.arrayCat(mag,0)

# We can then sort using various rows depending on whether we want the highest magnitude change, most recent, etc
# In this example, the sort row will be the magnitude, thus giving us the highest severity change
sorted = forSorting.arraySort(forSorting.arraySlice(0,1,2))

#### Inspect array outputs

Run the code below to add the layers to map to inspect them 

Double click to inpsect values

In [ ]:
# Set up map
Map.clearMap()
Map.addLayer(forSorting,{},'Array for Sorting')
Map.addLayer(sorted,{},'Sorted Array')
Map.turnOnInspector()
Map.view()

### Convert arrays into images
But those arrays still aren't immediately usable

Arrays will be converted into images representing loss magnitude and loss year

In [ ]:
# First, choose a change threshold
# Any changes more negative than this value will be flagged as loss
change_threshold = -0.15

In [ ]:
# Convert the sorted array image into a 2-band image
highest_mag_change = sorted.arraySlice(1,0,1).arrayProject([0]).arrayFlatten([['yr','mag']])

# Mask out any change 
highest_mag_change = highest_mag_change.updateMask(highest_mag_change.select(['mag']).lte(change_threshold))

# Pull the los magnitude palette and flip the color order
lossMagPalette = changeDetectionLib.lossMagPalette.split(',')
lossMagPalette.reverse()

# Set up map
Map.clearMap()
Map.addLayer(forSorting,{},'Array for Sorting',False)
Map.addLayer(sorted,{},'Sorted Array',False)
Map.addLayer(highest_mag_change.select(['mag']),{'min':-0.8,'max':-0.15,'palette':lossMagPalette},'Loss Magnitude')
Map.addLayer(highest_mag_change.select(['yr']),{'min':startYear,'max':endYear,'palette':changeDetectionLib.lossYearPalette},'Loss Year')
Map.turnOnInspector()
Map.view()

#### Using function

You can also do the same thing using a function: changeDetectionLib.convertToLossGain()

Array processing also happens within function in the geeViz libary, like [`changeDetectionLib.convertToLossGain()`]( https://github.com/redcastle-resources/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/changeDetectionLib.py#L778C5-L778C22)

You can see an example of how this is implemented in the geeViz [examples/LANDTRENDRViz.py](https://github.com/redcastle-resources/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/examples/LANDTRENDRViz.py)

In [ ]:
# use function to parse landtrendr array
lossGainDict = changeDetectionLib.convertToLossGain(lt_array, \
                                                      format = 'rawLandTrendr',\
                                                      lossMagThresh = -0.15,\
                                                      lossSlopeThresh = -0.1,\
                                                      gainMagThresh = 0.1,\
                                                      gainSlopeThresh = 0.1,\
                                                      slowLossDurationThresh = 3,
                                                      chooseWhichLoss = 'largest', 
                                                      chooseWhichGain = 'largest', 
                                                      howManyToPull = 1)

lossStack = lossGainDict['lossStack']
gainStack = lossGainDict['gainStack']

In [ ]:
#Set up viz params
vizParamsLossYear = {'min':startYear,'max':endYear,'palette':changeDetectionLib.lossYearPalette}
vizParamsLossMag = {'min':-0.8 ,'max':-0.15,'palette':list(reversed(changeDetectionLib.lossMagPalette.split(',')))}
  
vizParamsGainYear = {'min':startYear,'max':endYear,'palette':changeDetectionLib.gainYearPalette}
vizParamsGainMag = {'min':0.1,'max':0.8,'palette':changeDetectionLib.gainMagPalette}
  
vizParamsDuration = {'min':1,'max':5,'palette':changeDetectionLib.changeDurationPalette,'legendLabelLeftAfter':'year','legendLabelRightAfter':'years'}

# Select off the first change detected and visualize outputs
lossStackI = lossStack.select(['.*_1'])
gainStackI = gainStack.select(['.*_1'])
 
 # Set up map
Map.clearMap()
Map.addLayer(lossStackI.select(['loss_yr.*']),vizParamsLossYear,bandName +' Loss Year',True)
Map.addLayer(lossStackI.select(['loss_mag.*']),vizParamsLossMag,bandName +' Loss Magnitude',False)
Map.addLayer(lossStackI.select(['loss_dur.*']),vizParamsDuration,bandName +' Loss Duration',False)
  
Map.addLayer(gainStackI.select(['gain_yr.*']),vizParamsGainYear,bandName +' Gain Year',False)
Map.addLayer(gainStackI.select(['gain_mag.*']),vizParamsGainMag,bandName +' Gain Magnitude',False)
Map.addLayer(gainStackI.select(['gain_dur.*']),vizParamsDuration,bandName +' Gain Duration',False)
Map.turnOnInspector()
Map.view()


#### Using function## Run full LandTrendr - on all bands

Explain what each section of the code below is doing 
i.e., running LandTrendr for all bands and processing the outut into images that represent changes of interest

In [ ]:
# We will run LandTrendr for each band
#Clear the map in case it has been populated with layers/commands earlier
Map.clearMap()

# We can use any/all of the bands, but generally bands that use nir and swir are most useful
bandNames = ['red','nir','swir1','swir2','NBR','NDVI','brightness','greenness','wetness']

#Run LANDTRENDR
for bandName in bandNames:

  # Select the band and run LandTrendr
  run_params['timeSeries'] = composites.select([bandName])
  rawLT = ee.Algorithms.TemporalSegmentation.LandTrendr(**run_params)

  Map.addLayer(rawLT,{},'LT Raw {}'.format(bandName),False)

  # Notice the raw LandTrendr output is in GEE's image array format
  # We'll need to manipulate the raw output a bit to save on storage space
  
    # Mask out non vertex values to use less storage space
  ltArray = rawLT.select(['LandTrendr'])
  rmse = rawLT.select(['rmse'])
  vertices = ltArray.arraySlice(0,3,4)
  ltArray = ltArray.arrayMask(vertices)

  # Mask out all but the year and vertex fited values (get rid of the raw and vertex rows)
  ltArray = ltArray.arrayMask(ee.Image(ee.Array([[1],[0],[1],[0]])))
  rawLTForExport=ltArray.addBands(rmse)
  Map.addLayer(rawLTForExport,{},'LT Vertex Values Only {}'.format(bandName),False)

  # Show how the compressed vertex-only values can be decompressed later
  decompressedC = changeDetectionLib.simpleLTFit(ltArray,startYear,endYear,bandName,True,run_params['maxSegments'])
  Map.addLayer(decompressedC,{'bands':'{}_LT_fitted'.format(bandName),'min':0.2,'max':0.8},'Decompressed LT Output {}'.format(bandName),False)

  # Join the raw and fitted values
  fitted = decompressedC.select(['{}_LT_fitted'.format(bandName)])
  ltJoined = getImagesLib.joinCollections(composites.select([bandName]),fitted)
  Map.addLayer(ltJoined,{'bands':'{}_LT_fitted'.format(bandName),'min':0.2,'max':1,'palette':'D80,080'},'Raw and LT Fitted {}'.format(bandName),True)

  # Export LT array image
  # Set some properties that will be uses later
  rawLTForExport = rawLTForExport.set({'startYear':startYear,
                                          'endYear':endYear,
                                          'startJulian':startJulian,
                                          'endJulian':endJulian,
                                          'band':bandName})
  rawLTForExport =rawLTForExport.set(run_params)
  exportName = 'LT_Raw_{}_yrs{}-{}_jds{}-{}'.format(bandName,startYear,endYear,startJulian,endJulian)
  exportPath = export_landTrendr_collection + '/'+ exportName
  # Export output
  getImagesLib.exportToAssetWrapper(rawLTForExport,exportName,exportPath,{'.default':'sample'},studyArea,scale,crs,transform,overwrite=False)

Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

In [ ]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
# If you'd like to track the tasks, use this:
# tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(export_landTrendr_collection, type = 'imageCollection')

print('done')

Array processing also happens within functions 

`changeDetectionLib.convertToLossGain()` - https://github.com/redcastle-resources/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/changeDetectionLib.py#L778C5-L778C22



### Convert LandTrendr array into time series - for input into LCMS

https://github.com/redcastle-resources/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/changeDetectionLib.py#L565




In [ ]:
`# While we can use the LandTrendr output for change detection, LCMS uses it as inputs to change detection, land cover, and land use models
# This will show how the raw LandTrendr array image asset is converted into a time series 
# of annual fitted, segment duration, segment magnitude of change, and slope values
Map.clearMap()

lt_asset = ee.ImageCollection(export_landTrendr_collection)
# Convert stacked outputs into collection of fitted, magnitude, slope, duration, etc values for each year
# While the fitted LandTrendr value is generally of most importance to our models, 
# LandTrendr segment duration, slope, and magnitude of change can also help our models
lt_fit = changeDetectionLib.batchSimpleLTFit(lt_asset,startYear,endYear,None,bandPropertyName='band',arrayMode=True)

# Vizualize image collection that is used as predictors in LCMS models
# When you double-click the output on the map, notice the different values available to the models
Map.addLayer(lt_fit,{'bands':'swir2_LT_fitted,nir_LT_fitted,red_LT_fitted','min':0.15,'max':0.6},'LandTrendr All Predictors Time Series')


Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

#### Inspect
double click the map to query values and see actual and fitted values for indices over time

#### LandTrendr reduces noise in original composite tiem series 

The best way of understanding how LandTrendr contributes to reducing noise in the original composite time series is to visualize them side-by-side
This example takes the fitted values from LandTrendr and shows them along with the original
Notice many holes are now filled in by LandTrendr
In general the amount of noise is reduced. There is a risk however of fitting too much and omitting changes such as those seen in 2017 for Hurricane Maria

INCLUDE A SCREENSHOT OF HURRICANE MARIA

Click on the map, inspect the fitted time series, watch the composite timelapse and the Landtrendr timelapse

In [ ]:
Map.clearMap()

# The best way of understanding how LandTrendr contributes to reducing noise in the original composite time series
# is to visualize them side-by-side
# This example takes the fitted values from LandTrendr and shows them along with the original
# Notice many holes are now filled in by LandTrendr
# In general the amount of noise is reduced. There is a risk however of 
# fitting too much and omitting changes such as those seen in 2017 for Hurricane Maria

# Visualize fitted landTrendr composites
fitted_bns = lt_fit.select(['.*_fitted']).first().bandNames()
out_bns = fitted_bns.map(lambda bn: ee.String(bn).split('_').get(0))

# Give same names as composites
lt_synth = lt_fit.select(fitted_bns,out_bns)

# Visualize raw and LandTrendr fitted composites
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse,'Raw Composite Timelapse')

Map.addTimeLapse(lt_synth,getImagesLib.vizParamsFalse,'Fitted LandTrendr Composite Timelapse')


# Join the raw and fited values
ltJoined = getImagesLib.joinCollections(composites.select(bandNames),lt_fit.select(['.*_fitted']))

Map.addLayer(ltJoined,{'min':0.2,'max':1},'Raw and LT Fitted',True)


Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

### Vizualize other LandTrendr data

-- pull in example from https://github.com/redcastle-resources/geeViz/blob/master/examples/LANDTRENDRViz.py -- to show all LandTrendr outputs at the same time 

### Challenge: 

Calculate LandTrendr stats for NDVI and process arrays into most recent change year 

### Congratulations! You're done with module 3.1 

Other GeeViz LandTrendr examples: 
- https://github.com/redcastle-resources/geeViz/blob/master/examples/LANDTRENDRViz.py
- https://github.com/redcastle-resources/geeViz/blob/master/examples/LANDTRENDRWrapper.py
- https://github.com/redcastle-resources/geeViz/blob/master/examples/LANDTRENDRWrapperNotebook.ipynb


LandTrendr fitted data will be used as inputs to LCMS 